# LangChain Environment Setup

This notebook uses:
- Python 3.13
- LangChain
- LangChain OpenAI integration
- getpass (built-in Python module)


In [1]:
import getpass
import langchain
from langchain_openai import ChatOpenAI

print(f"LangChain version: {langchain.__version__}")
print(f"getpass module available: {getpass.__name__}")
print("Environment setup complete!")


LangChain version: 1.2.0
getpass module available: getpass
Environment setup complete!


In [ ]:
### Load model
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✓ OPENAI_API_KEY loaded from .env")
else:
    print("⚠ Warning: OPENAI_API_KEY not found in environment")

model = init_chat_model(model="gpt-4o-mini", temperature=0.1)


✓ OPENAI_API_KEY loaded from .env


In [8]:
### Create messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "do you help with homework?"},
    {"role": "assistant", "content": "Yes, I can help you with that."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "Paris is the capital of France."},
]

### Generate response
response = model.invoke(messages)
print(response.content)

The capital of France is Paris.


In [17]:
## Memory strategy, [LAST]

from langchain_core.messages import trim_messages

trim_messages(
    messages,
    max_tokens=30,
    strategy="last",
    token_counter=ChatOpenAI(model="gpt-4o-mini"),
    include_system=True
)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Paris is the capital of France.', additional_kwargs={}, response_metadata={})]

In [ ]:
# Prompt template

from langchain_core.prompts import ChatPromptTemplate

examples_prompt = ChatPromptTemplate.from_template("""
 tell me a joke about {topic}
""")

chain = examples_prompt | model

response = chain.invoke({"topic": "chickens"})
print(response.content)

Why did the chicken join a band?

Because it had the drumsticks!


In [ ]:
# chat prompt template

from langchain_core.prompts import ChatPromptTemplate

examples_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", 'tell me a joke about {topic}'),
])

chain = examples_prompt | model

response = chain.invoke({"topic": "cats"})
print(response.content)

Why was the cat sitting on the computer?

Because it wanted to keep an eye on the mouse!


In [ ]:
# with template placeholder

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

examples_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("msg"),
])

chain = examples_prompt | model

response = chain.invoke({"msg": [("user", "hola")]})
print(response.content)




¡Hola! ¿Cómo puedo ayudarte hoy?


In [52]:
# few shot prompt
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 🛩  1", "output": "1"},
    {"input": "1 🛩  2", "output": "-1"},
]

examples_prompt = ChatPromptTemplate.from_messages([
    ("human", '{input}'),
    ("ai", "{output}")    
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=examples_prompt,
        examples=examples
    )

main_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    few_shot_prompt,
    ("human", "{input}"),
])

chain = main_prompt | model

response = chain.invoke({"input": "10 🛩  5"})
print("token usage", response.response_metadata["token_usage"]["total_tokens"])
print(response.content)


token usage 58
5


In [55]:
# LCEL (LangChain Express Language)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("""
    traduce el siguiente texto al {lenguage}:
    {texto}
""")

parser = StrOutputParser()

chain = prompt | model | parser

response = chain.invoke({"lenguage": "español", "texto": "Hello, world!"})
print(response)



¡Hola, mundo!


In [3]:
manual_chat_memory = []

In [13]:
# Manual chat memory
from langchain_core.prompts import ChatPromptTemplate

user_input = input("Enter a message: ")
if len(manual_chat_memory) == 0:
    manual_chat_memory.append(("system", "tu eres un asistente de IA"))

manual_chat_memory.append(("user", user_input))

prompt = ChatPromptTemplate.from_messages(manual_chat_memory)

chain = prompt | model

response = chain.invoke({"input": user_input})
manual_chat_memory.append(("assistant", response.content))

In [14]:
for message in manual_chat_memory:
    print(message)


('system', 'tu eres un asistente de IA')
('user', 'waht is the bogota')
('user', 'what is tyou rname')
('assistant', "I am an AI assistant and don't have a personal name, but you can call me Assistant. How can I help you today?")
('user', 'i want to call you X')
('assistant', 'Sure, you can call me X! How can I assist you today?')
('user', 'is correct say what is the wheater in medellin?')
('assistant', 'The correct way to ask would be, "What is the weather in Medellín?" or "What\'s the weather like in Medellín?" If you need information about the weather, let me know!')
('user', 'are you X or Y')
('assistant', 'I am X, as you decided to call me that! How can I assist you further?')


In [ ]:
## Runnable, output parser and stremming
from langchain_core.runnables import RunnableLambda

sequence = RunnableLambda(lambda x: "response in english" if x.lower().startswith("u") else "response in spanish") | RunnableLambda(lambda x: x.upper()) | model

response = sequence.invoke("Hello, world!")
print(response.content)

¡Claro! Estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?


In [25]:
from langchain_core.runnables import RunnableLambda

sequence = RunnableLambda(lambda x: int(x)) | {
    "plus_one": RunnableLambda(lambda x: x+1),
    "plus_two": RunnableLambda(lambda x: x+2),
}

response = sequence.invoke("1")
print(response)

{'plus_one': 2, 'plus_two': 3}


In [60]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate


joke_query = "tell me a joke about"
parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "answer the user's question"),
    ("user", "{input} \n {format_instructions} \n"),
])

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

response = chain.invoke({"input": joke_query, "format_instructions": parser.get_format_instructions()})
print(response)



{'joke': 'Why did the developer go broke?', 'punchline': 'Because he used up all his cache!'}


In [61]:
import time

for s in chain.stream({"input": joke_query}):
    print(s)
    time.sleep(0.3)

{}
{'joke': ''}
{'joke': 'Why'}
{'joke': 'Why did'}
{'joke': 'Why did the'}
{'joke': 'Why did the developer'}
{'joke': 'Why did the developer go'}
{'joke': 'Why did the developer go broke'}
{'joke': 'Why did the developer go broke?'}
{'joke': 'Why did the developer go broke?', 'punchline': ''}
{'joke': 'Why did the developer go broke?', 'punchline': 'Because'}
{'joke': 'Why did the developer go broke?', 'punchline': 'Because he'}
{'joke': 'Why did the developer go broke?', 'punchline': 'Because he used'}
{'joke': 'Why did the developer go broke?', 'punchline': 'Because he used up'}
{'joke': 'Why did the developer go broke?', 'punchline': 'Because he used up all'}
{'joke': 'Why did the developer go broke?', 'punchline': 'Because he used up all his'}
{'joke': 'Why did the developer go broke?', 'punchline': 'Because he used up all his cache'}
{'joke': 'Why did the developer go broke?', 'punchline': 'Because he used up all his cache!'}


In [66]:
chunks = []
chain = prompt | model
async for chunk in model.astream(f"joke_query {"maths"}"):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)
    time.sleep(0.1)

Sure! Here’s a math joke for you:

Why was the equal sign so humble?

Because it knew it wasn't less than or greater than anyone else!